In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns



In [ ]:
class Model:
    
    def __init__(self, learning_rate, num_layers, size_layer):
		
        self.num_layers = num_layers
        self.size_layer = size_layer
        
        rnn_cells = tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)
        
        self.rnn_cells =  tf.nn.rnn_cell.MultiRNNCell([rnn_cells] * num_layers, state_is_tuple = False)
        
        self.X = tf.placeholder(tf.float32, (None, None, 1))
        self.Y = tf.placeholder(tf.float32, (None, None, 1))
        
        self.net_last_state = np.zeros((num_layers * 2 * size_layer))
        
        self.hidden_layer = tf.placeholder(tf.float32, shape=(None, num_layers * 2 * size_layer))
        
        self.outputs, self.last_state = tf.nn.dynamic_rnn(self.rnn_cells, self.X, initial_state = self.hidden_layer, dtype = tf.float32)
        
        self.rnn_W = tf.Variable(tf.random_normal((size_layer, 1)))
        self.rnn_B = tf.Variable(tf.random_normal((1,)))
            
        # linear dimension for x in (Wx + B)
        outputs_reshaped = tf.reshape(self.outputs, [-1, size_layer])
        
        y_batch_long = tf.reshape(self.Y, [-1, 1])
		
        # y = Wx + B
        self.logits = (tf.matmul(outputs_reshaped, self.rnn_W) + self.rnn_B)
        
        self.cost = tf.reduce_mean(tf.square(y_batch_long - self.logits))
        self.optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(self.cost)
		
    def step(self, sess, x, init_zero_state = True):
		
        # Reset the initial state of the network.
        if init_zero_state:
            init_value = np.zeros((self.num_layers * 2 * self.size_layer,))
        else:
            init_value = self.net_last_state
        
        # we want to get the constant in our output layer, same size as dimension input
        probs, next_lstm_state = sess.run([self.logits, self.last_state], feed_dict={self.X:[x], self.hidden_layer:[init_value]})

        self.net_last_state = next_lstm_state[0]

        return probs